In [1]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

In [72]:
video = cv2.VideoCapture(filename="videos/sleep_detection2.mp4")

width = 0
height = 0

#writer = cv2.VideoWriter(filename="videos/sleepd.mp4",
#                         fourcc=cv2.VideoWriter_fourcc(*"DIVX"),
#                         fps=50,
#                         frameSize=(360, 1180))

idList = [22, 23, 24, 26, 110, 157, 158, 160, 161, 130, 243]
color = (0, 0, 255)

detector = FaceMeshDetector()
plotY = LivePlot(w=540, h=360, yLimit=[10, 60])

ratioList = []

counter, blickCounter = 0, 0

while True:
    retval, img = video.read()
    if not retval: break
    
    
    
    img, faces = detector.findFaceMesh(img=img, draw=False)
    if faces:
        face = faces[0] #x
        for id in idList:
            cv2.circle(img=img,
                       center=face[id],
                       radius=5,
                       color=color,
                       thickness=cv2.FILLED)
            
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        
        lengthVertically, _ = detector.findDistance(p1=leftUp, p2=leftDown) # iki nokta arasındaki mesafeyi ölçüyoruz
        lengthHorizontally, _ = detector.findDistance(p1=leftLeft, p2=leftRight)
        
        cv2.line(img=img, pt1=leftUp, pt2=leftDown, color=(0, 255, 0), thickness=3)
        cv2.line(img=img, pt1=leftLeft, pt2=leftRight, color=(0, 255, 0), thickness=3)
        
        ratio = int((lengthVertically/lengthHorizontally) * 100)
        ratioList.append(ratio)
        
        if len(ratioList) > 3:
            ratioList.pop(0)
        
        ratioAvg = sum(ratioList) / len(ratioList)

        if ratioAvg < 35 and counter == 0:
            blickCounter += 1
            counter = 1
            color = (0, 255, 0)
        
        if counter != 0:
            counter += 1
            if counter > 10:
                counter = 0
                color = (0, 0, 255)
            
        cv2.putText(img=img,
                    text=f"Blink Counter: {blickCounter}",
                    org=(20, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 0, 255),
                    thickness=3)

        imgPlot = plotY.update(y=ratioAvg, color=color)
        img = cv2.resize(src=img, dsize=(640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    

    cv2.imshow(winname="Video", mat=imgStack)
    if cv2.waitKey(delay=50) == ord("q"): break
    
writer.release()
video.release()
cv2.destroyAllWindows()

In [63]:
?cvzone.stackImages

Signature: cvzone.stackImages(_imgList, cols, scale)
Docstring:
Stack Images together to display in a single window
:param _imgList: list of images to stack
:param cols: the num of img in a row
:param scale: bigger~1+ ans smaller~1-
:return: Stacked Image
File:      c:\users\murat\anaconda3\lib\site-packages\cvzone\utils.py
Type:      function
